In [66]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

testquery = """
SELECT *
FROM Facilities;
"""
df = pd.read_sql_query(testquery, engine)
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [67]:
query10 = """

SELECT sub.name, SUM(sub.ind_revenue) AS total_revenue
FROM
(SELECT slots, membercost, guestcost, name,
	CASE WHEN memid = '0' THEN slots*guestcost
	ELSE slots*membercost END AS ind_revenue
FROM Bookings
    INNER JOIN Facilities USING(facid)
    INNER JOIN Members USING(memid)) AS sub
GROUP BY sub.name
HAVING total_revenue < 1000
;

"""

df10 = pd.read_sql_query(query10, engine)
df10

,name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


In [68]:
query11 = """

SELECT M1.firstname AS Member_FirstName,
    M1.surname AS Member_LastName,
    M1.memid AS Member_ID,
    M2.firstname AS Recommender_FirstName,
    M2.surname AS Recommender_LastName,
    CAST(M2.memid AS CHAR) AS Recommender_ID
FROM Members as M1
    LEFT JOIN Members as M2
    ON M1.recommendedby = M2.memid
ORDER BY Member_LastName, Member_FirstName
;
"""

df11 = pd.read_sql_query(query11, engine)
df11

,Member_FirstName,Member_LastName,Member_ID,Recommender_FirstName,Recommender_LastName,Recommender_ID
0,Florence,Bader,15,Ponder,Stibbons,9
1,Anne,Baker,12,Ponder,Stibbons,9
2,Timothy,Baker,16,Jemima,Farrell,13
3,Tim,Boothe,8,Tim,Rownam,3
4,Gerald,Butters,5,Darren,Smith,1
5,Joan,Coplin,22,Timothy,Baker,16
6,Erica,Crumpet,36,Tracy,Smith,2
7,Nancy,Dare,7,Janice,Joplette,4
8,David,Farrell,28,None,None,None
9,Jemima,Farrell,13,None,None,None


In [69]:
query12 = """

SELECT name, SUM(slots) AS slots_usage
FROM(
    SELECT name, memid, slots
    FROM Bookings
        INNER JOIN Facilities
        USING(facid)
    WHERE memid <> 0)
GROUP BY name
ORDER BY slots_usage DESC;

"""

df12 = pd.read_sql_query(query12, engine)
df12

,name,slots_usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


In [70]:
query13 = """

SELECT name, Month, SUM(slots) AS slots_usage
FROM(
SELECT name, slots,
    CASE WHEN starttime LIKE '2012-07%' THEN 'July'
        WHEN starttime LIKE '2012-08%' THEN 'August'
        WHEN starttime LIKE '2012-09%' THEN 'September' END AS Month
FROM Bookings
    INNER JOIN Facilities
    USING(facid)
WHERE memid <> 0)
GROUP BY name, Month
ORDER BY slots_usage DESC
;
"""

df13 = pd.read_sql_query(query13, engine)
df13

,name,Month,slots_usage
0,Badminton Court,September,507
1,Pool Table,September,443
2,Tennis Court 1,September,417
3,Badminton Court,August,414
4,Tennis Court 2,September,414
5,Snooker Table,September,404
6,Massage Room 1,September,402
7,Table Tennis,September,400
8,Tennis Court 2,August,345
9,Tennis Court 1,August,339
